In [5]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_namespaced_pod("media-microservices")
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:


In [7]:
from kubernetes.client import CustomObjectsApi                                 
cust = CustomObjectsApi()
cust.list_cluster_custom_object('metrics.k8s.io', 'v1beta1', 'nodes')

{'kind': 'NodeMetricsList',
 'apiVersion': 'metrics.k8s.io/v1beta1',
 'metadata': {},
 'items': [{'metadata': {'name': 'minikube',
    'creationTimestamp': '2024-02-27T09:55:45Z',
    'labels': {'beta.kubernetes.io/arch': 'amd64',
     'beta.kubernetes.io/os': 'linux',
     'kubernetes.io/arch': 'amd64',
     'kubernetes.io/hostname': 'minikube',
     'kubernetes.io/os': 'linux',
     'minikube.k8s.io/commit': '0fb901cff6518c312b23f4ae34a9f955e8ef6ca1-dirty',
     'minikube.k8s.io/name': 'minikube',
     'minikube.k8s.io/primary': 'true',
     'minikube.k8s.io/updated_at': '2024_02_27T16_31_34_0700',
     'minikube.k8s.io/version': 'v1.32.0',
     'node-role.kubernetes.io/control-plane': '',
     'node.kubernetes.io/exclude-from-external-load-balancers': ''}},
   'timestamp': '2024-02-27T09:55:36Z',
   'window': '10.077s',
   'usage': {'cpu': '301060434n', 'memory': '829964Ki'}}]}

In [13]:
import pandas as pd
res = cust.list_cluster_custom_object('metrics.k8s.io', 'v1beta1', 'pods')
pod_metrics = res.get('items')
print(res)

{'kind': 'PodMetricsList', 'apiVersion': 'metrics.k8s.io/v1beta1', 'metadata': {}, 'items': [{'metadata': {'name': 'coredns-5dd5756b68-rhdfm', 'namespace': 'kube-system', 'creationTimestamp': '2024-02-27T09:56:34Z', 'labels': {'k8s-app': 'kube-dns', 'pod-template-hash': '5dd5756b68'}}, 'timestamp': '2024-02-27T09:56:33Z', 'window': '2.995s', 'containers': [{'name': 'coredns', 'usage': {'cpu': '1039398n', 'memory': '18948Ki'}}]}, {'metadata': {'name': 'etcd-minikube', 'namespace': 'kube-system', 'creationTimestamp': '2024-02-27T09:56:34Z', 'labels': {'component': 'etcd', 'tier': 'control-plane'}}, 'timestamp': '2024-02-27T09:56:33Z', 'window': '2.995s', 'containers': [{'name': 'etcd', 'usage': {'cpu': '12775292n', 'memory': '40552Ki'}}]}, {'metadata': {'name': 'kube-apiserver-minikube', 'namespace': 'kube-system', 'creationTimestamp': '2024-02-27T09:56:34Z', 'labels': {'component': 'kube-apiserver', 'tier': 'control-plane'}}, 'timestamp': '2024-02-27T09:56:33Z', 'window': '2.997s', 'con

In [14]:
import pandas as pd
res = cust.list_cluster_custom_object('metrics.k8s.io', 'v1beta1', 'pods')
pod_metrics = res.get('items')
metrics = []
for metric in pod_metrics:
    metrics.append([metric.get('metadata').get('name'), metric.get('containers')[0].get('usage').get('cpu'), metric.get('containers')[0].get('usage').get('memory')])
    # print("%s\t%s\t%s" % (metric.get('metadata').get('name'), metric.get('containers')[0].get('usage').get('cpu'), metric.get('containers')[0].get('usage').get('memory')))

df = pd.DataFrame(metrics, columns=["name", "cpu", "memory"])
print(df)

                               name        cpu    memory
0          coredns-5dd5756b68-rhdfm    956187n   18964Ki
1                     etcd-minikube   8291219n   40552Ki
2           kube-apiserver-minikube  20667670n  239476Ki
3  kube-controller-manager-minikube   6463903n   48276Ki
4                  kube-proxy-vxrh2     43492n   15788Ki
5           kube-scheduler-minikube    993981n   17040Ki
6   metrics-server-5696bdb95f-m2rhf    927090n   23232Ki
7               storage-provisioner    585145n   11936Ki


In [15]:
import sched, time
from IPython.display import clear_output
s = sched.scheduler(time.time, time.sleep)
def do_something(sc): 
    clear_output(wait=True)
    res = cust.list_cluster_custom_object('metrics.k8s.io', 'v1beta1', 'pods')
    pod_metrics = res.get('items')
    metrics = []
    for metric in pod_metrics:
        metrics.append([metric.get('metadata').get('name'), metric.get('containers')[0].get('usage').get('cpu'), metric.get('containers')[0].get('usage').get('memory')])
        # print("%s\t%s\t%s" % (metric.get('metadata').get('name'), metric.get('containers')[0].get('usage').get('cpu'), metric.get('containers')[0].get('usage').get('memory')))

    df = pd.DataFrame(metrics, columns=["name", "cpu", "memory"])
    print(df)
    s.enter(1, 1, do_something, (sc,))

s.enter(1, 1, do_something, (s,))
s.run()

Empty DataFrame
Columns: [name, cpu, memory]
Index: []


KeyboardInterrupt: 